Here I implement different kind of noises available in PennyLane library to simple example circuits to analyze their effects.

## 1. Classical parameter randomization

In [ ]:
!pip install pennylane

Performing a noisy X rotation with a random angle theta, then measuring Z.

In [2]:
import pennylane as qml
from pennylane import numpy as np

dev1 = qml.device("default.qubit", wires=1)

@qml.qnode(dev1)
def rot_circuit(prec):
    rand_angle = np.pi + prec*np.random.rand()
    qml.RX(rand_angle, wires=0)
    return qml.expval(qml.PauliZ(0))

We run the circuit with large random rotations 2 times to see the results.

In [3]:
rot_circuit(4)

tensor(0.07440198, requires_grad=True)

In [4]:
rot_circuit(4)

tensor(-0.58125668, requires_grad=True)

We see that for large random rotations, the expectation value of Z fluctuates wildly.

Now we run the circuit with low random rotations for 2 times.

In [5]:
rot_circuit(0.1)

tensor(-0.99576195, requires_grad=True)

In [6]:
rot_circuit(0.1)

tensor(-0.99978885, requires_grad=True)

We see that for small random rotations, the expectation value of Z doesn't change dramatically and changes to a small amount around -1.

## 2. BitFlip Noise
BitFlip noise applies an X operation with probability p and does nothing otherwise. This noise is applied to consider the effects of cosmic rays on terrestrial computations (sort of). Quantum noises like BitFlip are applied using channels.

In [7]:
#Preparing a device
dev2 = qml.device('default.mixed', wires=1)

In [8]:
#Preparing the BitFlip testing circuit
@qml.qnode(dev2)
def bitflip_circuit(p):
    qml.BitFlip(p, wires=0)
    return qml.expval(qml.PauliZ(0))

Running the circuit for 2 times with different probabilities to see the results.

In [9]:
bitflip_circuit(0.01)

array(0.98)

In [10]:
bitflip_circuit(0.99)

array(-0.98)

## 3. PhaseFlip Noise
BitFlip noise applies a Z operation with probability p and does nothing otherwise.
The phase flip impacts the relative phase between ∣0⟩ and ∣1⟩ but does not alter the probability distribution of these states.

In [26]:
#Preparing a device
dev3 = qml.device('default.mixed', wires=1)

In [30]:
@qml.qnode(dev3)
def phaseflip_circuit(p):
    qml.Hadamard(wires=0)  # Puts qubit in |+> state
    qml.PhaseFlip(p, wires=0)
    return qml.state()

In [31]:
phaseflip_circuit(0.01)

array([[0.5 +0.j, 0.49+0.j],
       [0.49+0.j, 0.5 +0.j]])

In [32]:
phaseflip_circuit(0.99)

array([[ 0.5 +0.j, -0.49+0.j],
       [-0.49+0.j,  0.5 +0.j]])

## 4. Amplitude Damping
This type of noise models the process where energy is lost from the qubit, often representing relaxation to the ground state ∣0⟩ due to environmental interactions.
It reduces the probability of measuring ∣1⟩ as the qubit relaxes to ∣0⟩, mimicking energy loss.

In [33]:
#Preparing a device
dev4 = qml.device('default.mixed', wires=1)

In [34]:
@qml.qnode(dev4)
def amplitude_damping_circuit(p):
    qml.PauliX(wires=0)  # Initialize the qubit to |1> state
    qml.AmplitudeDamping(p, wires=0)
    return qml.state()

In [35]:
amplitude_damping_circuit(0.01)

array([[0.01+0.j, 0.  +0.j],
       [0.  +0.j, 0.99+0.j]])

In [36]:
amplitude_damping_circuit(0.99)

array([[0.99+0.j, 0.  +0.j],
       [0.  +0.j, 0.01+0.j]])

## 5. Phase Damping
This type of noise causes the loss of quantum information without energy loss. It results in the decay of off-diagonal terms in the density matrix, affecting superposition coherence while leaving the population in ∣0⟩ and ∣1⟩ unchanged.
Phase damping reduces the visibility of interference in superposition states, which manifests as a loss of coherence in the system.

In [37]:
#Preparing a device
dev5 = qml.device('default.mixed', wires=1)

In [38]:
@qml.qnode(dev5)
def phase_damping_circuit(p):
    qml.Hadamard(wires=0)  # Puts qubit in |+> state
    qml.PhaseDamping(p, wires=0)
    return qml.state()

In [39]:
phase_damping_circuit(0.01)

array([[0.5       +0.j, 0.49749372+0.j],
       [0.49749372+0.j, 0.5       +0.j]])

In [40]:
phase_damping_circuit(0.99)

array([[0.5 +0.j, 0.05+0.j],
       [0.05+0.j, 0.5 +0.j]])

## 6. Depolarizing Channel
The depolarizing channel applies a probabilistic error where the qubit state is randomized to either the ∣0⟩ or ∣1⟩ state with equal probability, effectively introducing a "mixed" state. This is typically used to simulate arbitrary errors in a quantum system.
Depolarizing noise introduces uncertainty into the state, and the qubit will tend to a fully mixed state over multiple applications.

In [41]:
#Preparing a device
dev6 = qml.device('default.mixed', wires=1)

In [47]:
@qml.qnode(dev6)
def depolarizing_circuit(p):
    qml.Hadamard(wires=0)  # Puts qubit in |+> state
    qml.DepolarizingChannel(p, wires=0)
    return qml.state()

In [48]:
depolarizing_circuit(0.01)

array([[0.5       +0.j, 0.49333333+0.j],
       [0.49333333+0.j, 0.5       +0.j]])

In [49]:
depolarizing_circuit(0.99)

array([[ 0.5 +0.j, -0.16+0.j],
       [-0.16+0.j,  0.5 +0.j]])